<div style="background-color:#3c3d41;padding:30px;border-radius:5px">

<h1 style="font-size:22px;color:white;margin-top:0px">
How to download MODIS (MYD03, MOD03) Geolocation Files using python ?
</h1>

<p style="font-size:14px;color:white">
Author: Benjamin Marchant<br>
License: CC BY 4.0
</p>
  
</div>

In [1]:
from datetime import date
from os import path
from calendar import monthrange

import urllib.request
import urllib.request, json 
import pprint
import os

import glob
import netCDF4
import random
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import numpy.ma as ma

In [2]:
output_dir = '/Volumes/HD2/Datasets/Research/NASA'

To download a file from NASA LAADS DAAC, you need to create a token. Here are the steps to follow:

1. Visit this website: https://ladsweb.modaps.eosdis.nasa.gov/

2. Create a free account.

3. From the drop-down profile menu located in the top right corner, select "Generate Token."

4. Copy your generated token.



Please replace "the token below" with your own token. As an example, I have put in my own token. 



In [3]:
token = 'Ym1hcmNoYW50OlltVnVhbUZ0YVc0dWJXRnlZMmhoYm5SQWJtRnpZUzVuYjNZPToxNjI0NzE3MjEwOjEwZmNhNWU4ODVlNzc3OGUyMzE3NzFkZjNmNjUwMGIzNjVhMDY4ZWY'

In [4]:
opener = urllib.request.build_opener()
opener.addheaders = [('Authorization', 'Bearer {}'.format(token))]
urllib.request.install_opener(opener)

In [5]:
year = 2019
month = 8
day = 1

In [6]:
platform = 'AQUA'
instrument = 'MODIS'
product_category = 61
product = 'MYD03'

In [7]:
d0 = date(year, 1, 1)
d1 = date(year, month, day)
delta = d1 - d0

In [8]:
count_of_day = delta.days + 1

In [9]:
ladsweb_url = 'https://ladsweb.modaps.eosdis.nasa.gov/archive/allData/{}/{}/{:04d}/{:03d}.json'.format(product_category,product,year,count_of_day)

print(ladsweb_url)

https://ladsweb.modaps.eosdis.nasa.gov/archive/allData/61/MYD03/2019/213.json


In [10]:
with urllib.request.urlopen(ladsweb_url) as url:
    data = json.loads(url.read().decode())

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(data['content'][:10])

[   {   'archiveSets': 61,
        'cksum': '3110923774',
        'dataDay': '2019-213 = 2019-08-01',
        'downloadsLink': 'https://ladsweb.modaps.eosdis.nasa.gov/archive/allData/61/MYD03/2019/213/MYD03.A2019213.0000.061.2019213152538.hdf',
        'fileId': 5015657932,
        'md5sum': '06fa58bd59cc60040d79e3d77a821259',
        'mtime': 1564673173,
        'name': 'MYD03.A2019213.0000.061.2019213152538.hdf',
        'products': 'MYD03',
        'resourceType': 'File',
        'self': '/archive/allData/61/MYD03/2019/213/MYD03.A2019213.0000.061.2019213152538.hdf',
        'size': 31562876},
    {   'archiveSets': 61,
        'cksum': '2899861016',
        'dataDay': '2019-213 = 2019-08-01',
        'downloadsLink': 'https://ladsweb.modaps.eosdis.nasa.gov/archive/allData/61/MYD03/2019/213/MYD03.A2019213.0005.061.2019213153146.hdf',
        'fileId': 5015668333,
        'md5sum': '01d692c4220098c0954163627bc23fa8',
        'mtime': 1564673534,
        'name': 'MYD03.A2019213.0005.06

In [11]:
os.system( 'mkdir {}/{}'.format(output_dir,platform) )
os.system( 'mkdir {}/{}/{}'.format(output_dir,platform,instrument) )
os.system( 'mkdir {}/{}/{}/{}'.format(output_dir,platform,instrument,product))
os.system( 'mkdir {}/{}/{}/{}/{:04d}'.format(output_dir,platform,instrument,product,year) )
os.system( 'mkdir {}/{}/{}/{}/{:04d}/{:04d}_{:02d}_{:02d}'.format(output_dir,platform,instrument,product,year,year,month,day) )

mkdir: /Volumes/HD2/Datasets/Research/NASA/AQUA: File exists
mkdir: /Volumes/HD2/Datasets/Research/NASA/AQUA/MODIS: File exists


0

In [12]:
%%time

log_error = []

for file in data['content']:
    file_name = file['name']
    #print(file_name) 
    target_dir = '{}/{}/{}/{}/{:04d}/{:04d}_{:02d}_{:02d}/{}'.format(output_dir,platform,instrument,product,year,year,month,day,file_name)
    if not path.exists( target_dir ):
        try:
            ladsweb_url = 'https://ladsweb.modaps.eosdis.nasa.gov/archive/allData/{}/{}/{:04d}/{:03d}/{}'.format(product_category,product,year,count_of_day,file_name)
            urllib.request.urlretrieve(ladsweb_url,target_dir) 
        except:
            log_error.append(file)
            print('error')

CPU times: user 58.8 s, sys: 59.2 s, total: 1min 58s
Wall time: 17min 36s


In [13]:
file_list = glob.glob('{}/{}/{}/{}/{:04d}/{:04d}_{:02d}_{:02d}/*.hdf'.format(output_dir,platform,instrument,product,year,year,month,day))

len(file_list)

288

In [14]:
tot_size = 0

for file in file_list:
    file_size = os.path.getsize(file) * 0.001 # in KB
    tot_size += file_size
    print( str( round(file_size,2)  ) + ' KB')

31562.88 KB
32815.9 KB
29314.29 KB
26751.38 KB
25788.99 KB
25909.2 KB
26092.9 KB
25774.45 KB
27454.93 KB
29748.4 KB
35483.82 KB
30434.79 KB
34914.24 KB
37607.53 KB
33589.8 KB
35022.55 KB
29143.62 KB
27386.0 KB
28806.35 KB
29227.61 KB
31771.65 KB
32523.55 KB
27796.26 KB
27043.99 KB
26549.76 KB
26261.72 KB
26138.7 KB
26074.74 KB
27532.23 KB
37786.48 KB
35109.01 KB
30125.83 KB
32945.95 KB
31710.87 KB
29703.34 KB
28631.84 KB
26695.25 KB
27045.75 KB
28494.65 KB
29216.85 KB
32757.99 KB
31382.31 KB
28043.76 KB
26931.51 KB
30215.95 KB
29465.14 KB
26013.53 KB
27323.36 KB
36497.67 KB
43997.97 KB
32784.07 KB
31553.38 KB
29419.29 KB
26602.6 KB
26539.74 KB
27457.14 KB
38433.77 KB
33430.07 KB
30916.41 KB
30209.49 KB
32574.86 KB
31498.43 KB
27680.63 KB
27144.58 KB
32243.01 KB
29997.56 KB
28773.99 KB
39409.21 KB
42708.44 KB
40664.35 KB
29700.21 KB
33749.8 KB
32251.26 KB
27722.49 KB
26846.58 KB
35526.83 KB
34106.58 KB
30823.38 KB
30221.41 KB
30102.52 KB
32114.14 KB
31605.47 KB
27158.03 KB
26281.32 KB
2

In [15]:
tot_size

8891744.064000001

In [16]:
365 * 4 * 20 / ( 60 * 24 )

20.27777777777778

In [17]:
%%time

year = 2019
month = 8

platform = 'AQUA'
instrument = 'MODIS'
product_category = 61
product = 'MYD03'

log_error = []

for day in range(1, monthrange(year, month)[1]+1):
    
    d0 = date(year, 1, 1)
    d1 = date(year, month, day)
    delta = d1 - d0

    count_of_day = delta.days + 1

    ladsweb_url = 'https://ladsweb.modaps.eosdis.nasa.gov/archive/allData/{}/{}/{:04d}/{:03d}.json'.format(product_category,product,year,count_of_day)

    with urllib.request.urlopen(ladsweb_url) as url:
        data = json.loads(url.read().decode())

    os.system( 'mkdir {}/{}/{}/{}/{:04d}'.format(output_dir,platform,instrument,product,year) )
    os.system( 'mkdir {}/{}/{}/{}/{:04d}/{:04d}_{:02d}_{:02d}'.format(output_dir,platform,instrument,product,year,year,month,day) )
    
    for file in data['content']:
        file_name = file['name']
        #print(file_name) 
        target_dir = '{}/{}/{}/{}/{:04d}/{:04d}_{:02d}_{:02d}/{}'.format(output_dir,platform,instrument,product,year,year,month,day,file_name)
        if not path.exists( target_dir ):
            try:
                ladsweb_url = 'https://ladsweb.modaps.eosdis.nasa.gov/archive/allData/{}/{}/{:04d}/{:03d}/{}'.format(product_category,product,year,count_of_day,file_name)
                urllib.request.urlretrieve(ladsweb_url,target_dir) 
            except:
                log_error.append(file)

mkdir: /Volumes/HD2/Datasets/Research/NASA/AQUA/MODIS/MYD03/2019: File exists
mkdir: /Volumes/HD2/Datasets/Research/NASA/AQUA/MODIS/MYD03/2019/2019_08_01: File exists
mkdir: /Volumes/HD2/Datasets/Research/NASA/AQUA/MODIS/MYD03/2019: File exists
mkdir: /Volumes/HD2/Datasets/Research/NASA/AQUA/MODIS/MYD03/2019: File exists
mkdir: /Volumes/HD2/Datasets/Research/NASA/AQUA/MODIS/MYD03/2019: File exists
mkdir: /Volumes/HD2/Datasets/Research/NASA/AQUA/MODIS/MYD03/2019: File exists
mkdir: /Volumes/HD2/Datasets/Research/NASA/AQUA/MODIS/MYD03/2019: File exists
mkdir: /Volumes/HD2/Datasets/Research/NASA/AQUA/MODIS/MYD03/2019: File exists
mkdir: /Volumes/HD2/Datasets/Research/NASA/AQUA/MODIS/MYD03/2019: File exists
mkdir: /Volumes/HD2/Datasets/Research/NASA/AQUA/MODIS/MYD03/2019: File exists
mkdir: /Volumes/HD2/Datasets/Research/NASA/AQUA/MODIS/MYD03/2019: File exists
mkdir: /Volumes/HD2/Datasets/Research/NASA/AQUA/MODIS/MYD03/2019: File exists
mkdir: /Volumes/HD2/Datasets/Research/NASA/AQUA/MODIS

CPU times: user 30min 32s, sys: 28min 6s, total: 58min 38s
Wall time: 10h 21min 4s
